# Загружаем пакеты

In [ ]:
# загрузка пакетов: инструменты --------------------------------------------
# работа с массивами
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# для таймера
import time
# загрузка пакетов: модели -------------------------------------------------
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report
# константы
# ядро для генератора случайных чисел
my_seed = 9212
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np

# Загружаем данные

In [ ]:
# загружаем таблицу
url = "https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/winequality-red_for_lab.csv"
DF = pd.read_csv(url)
# оставим в таблице только указанные переменные
DF_raw = DF.filter(items = ['Y','fixed_acidity', 'volatile_acidity', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'pH', 'sulphates', 'alcohol'])
# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

In [ ]:
# первые строки
DF_raw.head()

In [ ]:
# типы столбцов
DF_raw.dtypes

Отложим 15% наблюдений для прогноза.

In [ ]:
DF = DF_raw.sample(frac = 0.85, random_state = my_seed)
DF_predict = DF_raw.drop(DF.index)

## Преобразование исходных данных и построение моделей

In [ ]:
DF.iloc[:, 2:].describe()

## Стандартизация и переход к главным компонентам

In [ ]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 2:].values)
# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',  np.around(frac_var_expl, 3), '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))


Таким образом, первые две главные компоненты объясняют 43.8% разброса 8 объясняющих
переменных.
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline
и оценим точность логистической регрессии с помощью перекрёстной проверки.

# Модель логистической регрессии с перекрёстной проверкой

In [ ]:
# данные для обучения моделей
X_train = DF.iloc[:, 1:]
y_train = DF.iloc[:, 0]
# объединяем в конвейер шкалирование, ГК с 4 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=4), LogisticRegression(random_state=my_seed, solver='lbfgs'))
# будем сохранять точность моделей в один массив:
score = list()
score_models = list()
# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=5,
 scoring='accuracy')
# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[0], ':', score[0])

# SVM с перекрёстной проверкой

In [ ]:
pipe_svc = make_pipeline(StandardScaler(),
 SVC(random_state=my_seed))
# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]
# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', refit=True, cv=kfold
                  )
gs = gs.fit(X_train, y_train)
# таймер
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность лучшей модели
np.around(gs.best_score_, 3)

In [ ]:
# параметры лучшей модели
# * ядерная функция
gs.best_estimator_.get_params()['svc__kernel']

In [ ]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

In [ ]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']


In [ ]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

# Прогноз на отложенные наблюдения по лучшей модели

In [ ]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

In [ ]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat = gs.best_estimator_.predict(X=DF_predict.iloc[:, 1:])
# точность
# характеристики точности
print(classification_report(DF_predict.iloc[:, 0], y_hat))